In [ ]:
#!pip install google-api-python-client pandas matplotlib

In [ ]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from googleapiclient.discovery import build

# Configurações da API
api_key = 'SUA_API_KEY_AQUI'
channel_id = 'ID_DO_SEU_CANAL'

youtube = build('youtube', 'v3', developerKey=api_key)

# Função para obter os últimos 42 domingos
def get_last_42_sundays():
    today = datetime.date.today()
    sundays = []
    while len(sundays) < 42:
        if today.weekday() == 6:  # Domingo é o dia 6
            sundays.append(today)
        today -= datetime.timedelta(days=1)
    return sundays

# Função para obter IDs de vídeos postados em datas específicas
def get_video_ids(channel_id, date):
    request = youtube.search().list(
        part='snippet',
        channelId=channel_id,
        publishedAfter=(date - datetime.timedelta(days=1)).isoformat() + 'T00:00:00Z',
        publishedBefore=date.isoformat() + 'T23:59:59Z',
        maxResults=50
    )
    response = request.execute()
    video_ids = [item['id']['videoId'] for item in response['items'] if item['id']['kind'] == 'youtube#video']
    return video_ids

# Função para obter as visualizações de um vídeo específico pelo ID
def get_video_views(video_id):
    request = youtube.videos().list(part='statistics', id=video_id)
    response = request.execute()
    view_count = int(response['items'][0]['statistics']['viewCount'])
    return view_count

# Função para obter dados de visualizações ao longo do tempo
def get_views_over_time(channel_id, sundays):
    data = {'Date': [], 'Views': []}
    for sunday in sundays:
        video_ids = get_video_ids(channel_id, sunday)
        if video_ids:
            views = sum(get_video_views(video_id) for video_id in video_ids)
        else:
            views = 0
        data['Date'].append(sunday.strftime('%Y-%m-%d'))
        data['Views'].append(views)
    return pd.DataFrame(data)

# Obter os últimos 42 domingos
sundays = get_last_42_sundays()

# Obter dados de visualizações
df = get_views_over_time(channel_id, sundays)

# Criar gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(df['Date'], df['Views'], color='blue')
plt.xlabel('Data')
plt.ylabel('Visualizações')
plt.title('Evolução das Visualizações da Live ao Longo do Tempo')
plt.xticks(rotation=45)
plt.tight_layout()

# Salvar o gráfico como uma imagem
plt.savefig('imagens/analise_live.png')

# Mostrar o gráfico no notebook
plt.show()